# Practical Example

You are given data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertizing to him/her. If we can focus our efforts ONLY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

You have a .csv summarizing the data. There are several variables: Customer ID, Book length in mins_avg (average of all purchases), Book length in minutes_sum (sum of all purchases), Price Paid_avg (average of all purchases), Price paid_sum (sum of all purchases), Review (a Boolean variable), Review (out of 10), Total minutes listened, Completion (from 0 to 1), Support requests (number), and Last visited minus purchase date (in days).

So these are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (so 0, or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

Good luck!

## Import libraries

In [5]:
import numpy as np
import tensorflow as tf

## Data

In [21]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load("Audiobooks_data_train.npz")
# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz["inputs"].astype(np.float64)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz["targets"].astype(np.int64)

# we load the validation data in the temporary variable
npz = np.load("Audiobooks_data_validation.npz")
validation_inputs = npz["inputs"].astype(np.float64)
validation_targets = npz["targets"].astype(np.int64)

npz = np.load("Audiobooks_data_test.npz")
test_inputs = npz["inputs"].astype(np.float64)
test_targets = npz["targets"].astype(np.int64)



## Model
Outline, optimizers, loss, early stopping and training

In [23]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation funct
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation="softmax")
])

### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration

model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epoch = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)


# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(
    train_inputs, 
    train_targets, 
    batch_size = batch_size, 
    epochs = max_epoch,
    callbacks=[early_stopping],
    validation_data = (validation_inputs, validation_targets),
    verbose = 2)

Epoch 1/100
36/36 - 1s - loss: 0.6277 - accuracy: 0.6633 - val_loss: 0.5267 - val_accuracy: 0.7629 - 808ms/epoch - 22ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4914 - accuracy: 0.7631 - val_loss: 0.4325 - val_accuracy: 0.7852 - 50ms/epoch - 1ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4272 - accuracy: 0.7812 - val_loss: 0.3799 - val_accuracy: 0.8188 - 50ms/epoch - 1ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3936 - accuracy: 0.7960 - val_loss: 0.3576 - val_accuracy: 0.8188 - 51ms/epoch - 1ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3771 - accuracy: 0.8008 - val_loss: 0.3421 - val_accuracy: 0.8210 - 53ms/epoch - 1ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3678 - accuracy: 0.8058 - val_loss: 0.3406 - val_accuracy: 0.8255 - 48ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3609 - accuracy: 0.8053 - val_loss: 0.3273 - val_accuracy: 0.8658 - 52ms/epoch - 1ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3582 - accuracy: 0.8092 - val_loss: 0.3297 - val_accuracy: 0.8479 - 50ms/epoch - 1ms/step
Epoch 9/100
36

## Test

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset.

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [24]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

print("Test loss: {0:.2f}, Test accuracy: {1:.2f}%".format(test_loss, test_accuracy * 100))

14/14 [==============================] - 0s 949us/step - loss: 0.3535 - accuracy: 0.8125
Test loss: 0.35, Test accuracy: 81.25%


Using the initial model and hyperparameters given in this notebook, the final test accuracy should be roughly around 81%.

Note that each time the code is rerun, we get a different accuracy because each training is different. 

We have intentionally reached a suboptimal solution, so you can have space to build on it!